# Photogeo→CSV→MAPPING
[Photogeoリポジトリ](https://github.com/pygeosham/photogeo)で示された手順で取得されるデータ(Google スプレッドシート)をCSVファイルとして読み込んでJupyterLiteで表示させるサンプルコードです

地図ライブラリのインポート

Folium (leaflet.jsを用いた地図表示ライブラリ) のインストール

In [ ]:
%pip install -q folium

以下の情報が格納されているCSVファイル(photo_data.csv)を読み込みデータフレームに変換
+ Forms経由でGoogleDriveに格納された画像情報へのリンク(画像ファイルの指定 列)
+ Forms経由で入力された画像についてのコメント(タイトル,解説 列)
+ 画像から抽出された十進緯度経度の位置情報(lat,lon 列)

In [ ]:
import pandas as pd
df = pd.read_csv("photo_data.csv")
df

読み込んだ情報をFoliumを用いて地図上にプロット
+ 背景はOpenStreetMap
+ マーカをクリックすると画像のプレビューとタイトルがポップアップする

In [ ]:
import folium
# シンプルな地図を用意

map0 = folium.Map(
   # 地図の初期位置を決める(十進緯度経度で都庁の位置を指定)
   location=[35.708886,139.710989],
   # ズームレベルの初期値
   zoom_start = 12,
   # 地図の種類の選択
   tiles = "OpenStreetMap"
)

# 指定した位置にマーカーを置くコード(とりあえずいろいろデフォルトのまま)
for i in range(len(df)):
    #画像preview用のURLの生成
    #Formsにアップした画像が格納されている列名を指定する(ここでは"画像ファイルの指定")
    #googledrive上のパスを文字列として抽出
    s = df.iloc[i]["画像ファイルの指定"]
    
    # =で文字列を分割することで 画像ID を抽出し
    #画像Previewに用いるベースのURL文字列に結合することで
    #Preview用のURLを生成する
    # 2024年02月時点で以下の記載でポップアップのプレビュー表示が可能
    # (たまにgoogledriveの仕様が変わるので注意)
    sep = '='
    imgview = "https://lh3.googleusercontent.com/d/" + s.split(sep)[1]
    folium.Marker(
        # lat lon の順序でリストで緯度経度を指定
        # 緯度と経度が格納されている列名を指定する(ここでは "lat" "lon")
        location = [df.iloc[i]["lat"], df.iloc[i]["lon"]],
        
        # マウスオーバー時に表示される列名を指定(ここではタイトル)
        tooltip = str(df.iloc[i]["タイトル"]),
        
        # ポップアップ表示ではタイトル,画像へのリンク,画像のpreviewを表示
        popup = '<table width="150"><tr><th><a href="' 
                 + str(df.iloc[i]["画像ファイルの指定"])
                 + '" target="_blank">' 
                 + str(df.iloc[i]["タイトル"]) 
                 + '</a></th></tr>'
                 + '<tr><td><img src="' + imgview + '" width="150"></td><tr/></table>', 
    ).add_to(map0)

# ブラウザで開くと同じ地図が表示されるhtmlファイルとして出力(保存)
#map0.save("map.html")

# 地図をjupyterlab上で描画する
map0